# Preprocessing notebook

The goal of this notebook is to to process the data obtained from the ResNets and the extracted feature vectors. The main steps performed are creating positive and negative samples. While there was some experimentation with dimensionality reduction, there was no performance gain from it.

## Imports

In [369]:
import pandas as pd
import numpy as np
import json

### Data imports

In [370]:
df = pd.read_json('../data/tracks.json')
with open('../data/labels.json') as f:
    labels = json.load(f)

### Visualizating the data

In [371]:
df

,track1,track2,person1,person2,version,episode,segment,features,video
0,412,411,phoebe,gunther,0,episode01,"[1, 16]","[0.305748283863067, 0.403462648391723, 0.37505...",video412_411_0.mp4
1,412,411,phoebe,gunther,0,episode01,"[17, 32]","[0.9845066666603081, 0.00869304779917, 0.28160...",video412_411_0.mp4
2,412,411,phoebe,gunther,0,episode01,"[33, 48]","[0.601864755153656, 0.16546161472797302, 1.026...",video412_411_0.mp4
3,412,411,phoebe,gunther,0,episode01,"[49, 64]","[0.8926258087158201, 0.007743219844996, 1.2013...",video412_411_0.mp4
4,73,74,phoebe,rachel,0,episode01,"[1, 16]","[0.28641876578330905, 0.00654274970293, 1.8379...",video073_074_0.mp4
...,...,...,...,...,...,...,...,...,...
19343,558,557,monica,phoebe,0,episode14,"[49, 64]","[0.022156653925776003, 0.604685246944427, 0.00...",video558_557_0.mp4
19344,549,547,monica,joey,0,episode14,"[1, 16]","[1.074834704399108, 0.797054231166839, 2.67562...",video549_547_0.mp4
19345,549,547,monica,joey,0,episode14,"[17, 32]","[0.794210314750671, 0.7135680913925171, 1.6077...",video549_547_0.mp4
19346,549,547,monica,joey,0,episode14,"[33, 48]","[1.175412058830261, 1.063856244087219, 1.45035...",video549_547_0.mp4


The _version_ column is superfluous so we drop it.

In [234]:
df.drop(columns=['version'], inplace=True)

### Track example

Let us look at the track 16/17 in episode 1. This corresponds to the segments [1, 16] to [97, 112] of Chandler and Phoebe. We see that the associated label is False, i.e. there is no interaction.

In [237]:
df[(df['track1'] == 16) & (df['track2'] == 17) & (df['episode'] == 'episode01')]

,track1,track2,person1,person2,episode,segment,features,video
587,16,17,chandler,phoebe,episode01,"[1, 16]","[0.35464870929718, 1.900528669357299, 1.825586...",video016_017_0.mp4
588,16,17,chandler,phoebe,episode01,"[17, 32]","[0.009576153010129, 0.39936581254005404, 1.336...",video016_017_0.mp4
589,16,17,chandler,phoebe,episode01,"[33, 48]","[0.06511735171079601, 0.9418035745620721, 0.05...",video016_017_0.mp4
590,16,17,chandler,phoebe,episode01,"[49, 64]","[0.24777355790138203, 2.288475275039673, 1.437...",video016_017_0.mp4
591,16,17,chandler,phoebe,episode01,"[65, 80]","[0.33924889564514105, 1.943073034286499, 1.422...",video016_017_0.mp4
592,16,17,chandler,phoebe,episode01,"[81, 96]","[0.19470424950122803, 2.206629037857055, 0.656...",video016_017_0.mp4
593,16,17,chandler,phoebe,episode01,"[97, 112]","[0.23017062246799402, 1.040428400039672, 1.093...",video016_017_0.mp4


In [322]:
df[(df['track1'] == 17) & (df['track2'] == 16) & (df['episode'] == 'episode01')]

,track1,track2,person1,person2,episode,segment,features,video
979,17,16,phoebe,chandler,episode01,"[1, 16]","[0.019588408991694003, 0.625551521778106, 3.55...",video017_016_0.mp4
980,17,16,phoebe,chandler,episode01,"[17, 32]","[0.006818289868533001, 0.7460855245590211, 3.5...",video017_016_0.mp4
981,17,16,phoebe,chandler,episode01,"[33, 48]","[0.300980627536773, 0.442733347415924, 2.75530...",video017_016_0.mp4
982,17,16,phoebe,chandler,episode01,"[49, 64]","[0.111174792051315, 1.04053258895874, 3.037196...",video017_016_0.mp4
983,17,16,phoebe,chandler,episode01,"[65, 80]","[0.087654069066047, 1.101483702659607, 3.49819...",video017_016_0.mp4
984,17,16,phoebe,chandler,episode01,"[81, 96]","[0.022392775863409, 0.8720298409461971, 3.5221...",video017_016_0.mp4
985,17,16,phoebe,chandler,episode01,"[97, 112]","[0.442525416612625, 0.9762797951698301, 2.9311...",video017_016_0.mp4


In [323]:
for label in labels['episode01']:
    if (label[0], label[1]) in {(16, 17), (17, 16)}:
        print(label)

[16, 17, False]


This is an example of a track from which we could create type 1 negatives. 

## Creating positive examples

We loop over the labels and fetch the corresponding tracks.

In [240]:
def create_positive_instances(data, labels):
    """
    Create positive instances from the data.
    """
    positives = []
    for episode in labels:
        for track1, track2, label in labels[episode]:
            if label is True:
                person1_features = data[(data['track1'] == track1) & (data['track2'] == track2) & (data['episode'] == episode)].sort_values(by='segment', key=lambda col: col.map(lambda seg: seg[0]))
                person2_features = data[(data['track1'] == track2) & (data['track2'] == track1) & (data['episode'] == episode)].sort_values(by='segment', key=lambda col: col.map(lambda seg: seg[0]))
                assert len(person1_features) == len(person2_features)
                for person1_feature, person2_feature in zip(person1_features.itertuples(), person2_features.itertuples()):
                    positives.append({
                        # 'episode': episode,
                        # 'track1': track1,
                        # 'track2': track2,
                        # 'segment': person1_feature.segment,
                        'person1_features': person1_feature.features,
                        'person2_features': person2_feature.features
                    })
                
    return pd.DataFrame(positives)

In [241]:
df_pos = create_positive_instances(df, labels)
print(f'Number of positive instances: {len(df_pos)}')

Number of positive instances: 7514


## Creating negatives

At this stage, seeing that we have 7514 samples, we will only create "type 4" negative samples as they will be the most powerful and we can essentially create an infinite amount of them. The code for creating type 1 negatives is similar to the code for creating positive samples.

### Type 1 negatives

These are negatives from matching spatio-temporal windows but where we know the pair not be interacting.

In [243]:
def create_type1_negatives(data, labels):
    """
    Create type 1 negatives from matching spatio-temporal features.
    """
    negatives = []
    for episode in labels:
        for track1, track2, label in labels[episode]:
            if label is False:
                person1_features = data[(data['track1'] == track1) & (data['track2'] == track2) & (data['episode'] == episode)].sort_values(by='segment', key=lambda col: col.map(lambda seg: seg[0]))
                person2_features = data[(data['track1'] == track2) & (data['track2'] == track1) & (data['episode'] == episode)].sort_values(by='segment', key=lambda col: col.map(lambda seg: seg[0]))
                assert len(person1_features) == len(person2_features)
                for person1_feature, person2_feature in zip(person1_features.itertuples(), person2_features.itertuples()):
                    negatives.append({
                        # 'episode': episode,
                        # 'track1': track1,
                        # 'track2': track2,
                        # 'segment': person1_feature.segment,
                        'person1_features': person1_feature.features,
                        'person2_features': person2_feature.features
                    })
                
    return pd.DataFrame(negatives)

In [244]:
df_neg1 = create_type1_negatives(df, labels)
print(f'Number of type 1 negative instances: {len(df_neg1)}')

Number of type 1 negative instances: 498


### Type 2 negatives

These are negatives from different spatio-temporal windows taken from the same track. The implementation is not complete as in the end we did  need them.

In [245]:
# FIXME:
# def create_type2_negatives(data, labels):

### Type 4 negatives

These are negatives from different spatio-temporal windows and from different episodes and different characters. This creates a large number of negatives. We fix the number of negatives to be created as an argument in the function call. Given that we have 7514 samples, we will create a total of 100_000 for good measure as this seems reasonable. We can always reduce the number of negatives when training the model.

In [372]:
def create_type4_negatives(df: pd.DataFrame, labels, negatives_number=10_000):
    """Create type 4 negatives from different episodes and characters"""
    negatives = []
    while len(negatives) < negatives_number:
        episode1, episode2 = np.random.choice(list(labels.keys()), size=2, replace=False)
        tracki, trackj = df[df['episode'] == episode1].sample().iloc[0], df[df['episode'] == episode2].sample().iloc[0]
        if tracki['person1'] != trackj['person1']:
            negatives.append({
                'episode_1': tracki['episode'],
                'episode_2': trackj['episode'],
                'person_1': tracki['person1'],
                'person_2': trackj['person1'],
                'version_1': tracki['version'],
                'version_2': trackj['version'],
                'tracki1': tracki['track1'],
                'tracki2': tracki['track2'],
                'trackj1': trackj['track1'],
                'trackj2': trackj['track2'],
                'segmenti': tracki['segment'],
                'segmentj': trackj['segment'],
                'person1_features': tracki['features'],
                'person2_features': trackj['features']
            })
        if len(negatives) % 10_000 == 0:
            print(f'Negatives created: {len(negatives)}')
    return pd.DataFrame(negatives)

In [247]:
df_neg4 = create_type4_negatives(df, labels, negatives_number=100_000)
print(f'Number of type 4 negative instances: {len(df_neg4)}')
df_neg4

Negatives created: 0
Negatives created: 10000
Negatives created: 20000
Negatives created: 30000
Negatives created: 30000
Negatives created: 40000
Negatives created: 50000
Negatives created: 60000
Negatives created: 70000
Negatives created: 80000
Negatives created: 90000
Negatives created: 100000
Number of type 4 negative instances: 100000


,person1_features,person2_features
0,"[0.5935591459274291, 2.136274099349975, 0.2862...","[0.33274304866790705, 0.5014960169792171, 0.07..."
1,"[0.0018671104917300002, 0.9743065834045411, 2....","[0.016910068690776003, 1.055215001106262, 0.14..."
2,"[1.045803308486938, 0.0, 0.20327922701835602, ...","[1.455870866775512, 0.8921209573745721, 0.0077..."
3,"[0.38271686434745705, 1.103181362152099, 0.056...","[0.0, 0.111097387969493, 0.21661548316478701, ..."
4,"[0.053021796047687, 1.330795764923095, 1.25090...","[0.214696615934371, 0.5652663707733151, 0.0, 0..."
...,...,...
99995,"[0.042366411536931006, 0.6765475869178771, 0.7...","[0.20562626421451502, 0.9625936746597291, 1.12..."
99996,"[0.319629520177841, 2.245369434356689, 0.40146...","[0.150232255458831, 1.719058871269226, 0.0, 0...."
99997,"[0.09444165974855401, 0.9858267903327941, 0.08...","[1.3766940832138062, 0.44322270154953003, 0.03..."
99998,"[0.0, 0.0, 0.0, 0.029769795015454, 0.048692673...","[2.134280681610107, 0.49204528331756503, 0.036..."


## Concatenation of the positive and negative examples

In [316]:
df_pos['label'] = 1
df_neg4['label'] = 0
df_out = pd.concat([df_pos[['person1_features', 'person2_features', 'label']], df_neg4[['person1_features', 'person2_features', 'label']]], axis=0, ignore_index=True)

In [317]:
df_out

,person1_features,person2_features,label
0,"[0.073240458965301, 0.108803056180477, 1.90936...","[1.012074708938598, 0.40622571110725403, 0.160...",1
1,"[0.21179485321044902, 0.012390901334583001, 1....","[0.771273374557495, 0.16172519326210003, 0.277...",1
2,"[0.010071376338601001, 0.11898501217365201, 1....","[0.579812049865722, 0.25832888484001104, 0.559...",1
3,"[0.371864974498748, 0.019903786480426, 0.49261...","[0.13664688169956202, 0.9641482234001161, 0.31...",1
4,"[0.24769486486911702, 0.0, 0.562934577465057, ...","[0.6391886472702021, 0.071292459964752, 2.6756...",1
...,...,...,...
107509,"[0.042366411536931006, 0.6765475869178771, 0.7...","[0.20562626421451502, 0.9625936746597291, 1.12...",0
107510,"[0.319629520177841, 2.245369434356689, 0.40146...","[0.150232255458831, 1.719058871269226, 0.0, 0....",0
107511,"[0.09444165974855401, 0.9858267903327941, 0.08...","[1.3766940832138062, 0.44322270154953003, 0.03...",0
107512,"[0.0, 0.0, 0.0, 0.029769795015454, 0.048692673...","[2.134280681610107, 0.49204528331756503, 0.036...",0


## Normalization

We apply L2 normalization to the data.

In [318]:
# df_out[['person1_features', 'person2_features']] = df_out[['person1_features', 'person2_features']].applymap(lambda x: np.array(x) / (np.linalg.norm(np.array(x))))

## Saving data

In [319]:
from sklearn.utils import shuffle

In [320]:
df_out.to_json('../data/features.json', orient='records')

## Dimensionality reduction

This section of code has been commented out as it is no longer used.

In [ ]:
# df_new = pd.DataFrame(index=np.arange(len(df_out)), columns=np.arange(1025))

In [ ]:
# for row in df_out.itertuples(index=True):
#     df_new.loc[row[0]] = row[1] + row[2] + [row[3]]

In [ ]:
# df_new

In [ ]:
# from sklearn.decomposition import PCA
# from sklearn.preprocessing import StandardScaler
# from sklearn.manifold import TSNE

In [ ]:
# df_out
# x = []
# for row in df_out.itertuples():
#     x.append(row[1])
#     x.append(row[2])
# x = np.array(x)
# x.shape
# # x = np.array([[np.array(row[0]), np.array(row[1])] for row in df_out.itertuples()])
# # x.shape

In [ ]:
# pca = PCA(n_components=100)

# # x = df_new.drop(columns=[1024])
# x = StandardScaler().fit_transform(x)
# # x = pca.fit_transform(x)
# x = TSNE(n_components=2).fit_transform(x)
# x.shape

In [ ]:
# y = []
# for i in range(len(x) // 2):
#     y.append([x[2*i], x[2*i+1]])
# principal_df = pd.DataFrame(data=y, columns=['principal component 1', 'principal component 2'])
# final_df = pd.concat([principal_df, df_out[['label']]], axis=1)

In [ ]:
# final_df.rename(columns={1024: 'label', 'principal component 1': 'person1_features', 'principal component 2': 'person2_features'}, inplace=True)
# final_df